In [1]:
import os 
import pandas as pd
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from data_handler.preprocessing import FinancialDataPreprocessor
from sentiment.model import SentimentPriceModel
from data_handler.models import Asset
from sentiment.models import Category

In [6]:
econwords = Category.objects.get(name="Econ@").words.all().values_list('word', flat=True)
negwords = Category.objects.get(name="Negativ").words.all().values_list('word', flat=True)
legal = set(Category.objects.get(name="Legal").words.all().values_list('word', flat=True))
polit = set(Category.objects.get(name="Polit@").words.all().values_list('word', flat=True))
socrel = set(Category.objects.get(name="SocRel").words.all().values_list('word',flat=True))
econwords = set(econwords)
negwords = set(negwords)
negecon = econwords.intersection(negwords)
legneg = legal.intersection(negwords)
politneg = polit.intersection(negwords)
total = negecon.union(politneg).union(legneg).union(socrel)

spm = SentimentPriceModel()
spm.get_asset(asset='^FTMC', column_name='FTSE 250', zscore=True)
spm.add_sentiment_variable(category='PolitNegEcon', set=True, column_name='Special Negative Words', zscore=True, sentiment_words=list(total))


,Special Negative Words
date,
2016-01-04,-1.326730
2016-01-05,-1.361495
2016-01-06,-1.030069
2016-01-07,-1.286171
2016-01-08,-1.326730
...,...
2019-12-23,-1.229388
2019-12-24,-1.121617
2019-12-27,-0.332452


In [11]:
mv = spm.multivariate_df
for col in mv.columns:
    mv[col] = mv.loc[:, col].rolling(window=1).mean()
mv = mv.dropna()
mp = spm.produce_plot_model(df=mv)
mp.plot()


In [15]:
results = spm.var_model(lag=4, freq="B")
results.summary()

INFO - 17:22:29: Period type already set to <BusinessDay>


  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Sat, 28, Mar, 2020
Time:                     17:22:29
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                  -0.869786
Nobs:                     1004.00    HQIC:                 -0.924383
Log likelihood:          -2350.39    FPE:                   0.383719
AIC:                    -0.957845    Det(Omega_mle):        0.376931
--------------------------------------------------------------------
Results for equation FTSE 250
                               coefficient       std. error           t-stat            prob
--------------------------------------------------------------------------------------------
const                             0.060639         0.026428            2.294           0.022
L1.FTSE 250                       0.085702         0.031473            2.723           0.006
L1.Specia